
#  Benin (Malanville) — Exploratory Data Analysis (EDA)

### 📘 Overview
This notebook performs **Task 2: Data Profiling, Cleaning, and Exploratory Data Analysis (EDA)** for the Benin solar dataset (`benin-malanville.csv`).  


**Objectives:**
- Profile and clean the dataset.
- Detect missing values and outliers.
- Explore relationships between solar and weather parameters.
- Export a cleaned version ready for comparison across regions.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.dates import DateFormatter
import os

%matplotlib inline
sns.set_style("whitegrid")
print("✅ Libraries imported successfully.")

In [ ]:
# Load dataset
csv_path = r"/data/benin-malanville.csv"
df = pd.read_csv(csv_path, low_memory=False)

# Try to detect timestamp column automatically
for col in ['timestamp', 'Timestamp', 'time', 'Time', 'datetime', 'Datetime']:
    if col in df.columns:
        df['Timestamp'] = pd.to_datetime(df[col], errors='coerce')
        break

print("✅ Data loaded successfully!")
print(f"Shape: {df.shape}")
df.head()

In [ ]:
# Display dataset info and basic summary
df.info()
display(df.describe(include='all').transpose().head(15))

In [ ]:
# Missing-value report
missing = df.isna().sum().to_frame('Missing Values')
missing['% Missing'] = (missing['Missing Values'] / len(df)) * 100
display(missing.sort_values('% Missing', ascending=False).head(20))

cols_over_5pct = missing[missing['% Missing'] > 5].index.tolist()
print("Columns with >5% missing values:", cols_over_5pct)

In [ ]:
# Outlier detection using Z-scores
core_cols = [c for c in ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust'] if c in df.columns]
outliers = pd.DataFrame(False, index=df.index, columns=core_cols)

for c in core_cols:
    z = np.abs(stats.zscore(df[c].fillna(df[c].median())))
    outliers[c] = z > 3

df['Outlier'] = outliers.any(axis=1)
print("Total outlier rows:", df['Outlier'].sum())
df.loc[df['Outlier']].head()

In [ ]:
# Cleaning: Drop invalid timestamps and impute missing values with median
cleaned = df.copy()
cleaned = cleaned[~cleaned['Timestamp'].isna()].copy()

impute_cols = [c for c in ['GHI','DNI','DHI','ModA','ModB','WS','WSgust','Tamb','RH','BP'] if c in cleaned.columns]
for c in impute_cols:
    cleaned[c].fillna(cleaned[c].median(), inplace=True)

os.makedirs(os.path.dirname(r"/mnt/data/data/benin_clean.csv"), exist_ok=True)
cleaned.to_csv(r"/mnt/data/data/benin_clean.csv", index=False)
print("✅ Cleaned data saved to:", r"/mnt/data/data/benin_clean.csv")

In [ ]:
# Time series analysis
plt.figure(figsize=(12, 5))
for col in [c for c in ['GHI','DNI','DHI','Tamb'] if c in cleaned.columns]:
    plt.plot(cleaned['Timestamp'], cleaned[col], label=col)
plt.legend()
plt.title("Time Series of Solar and Temperature Data")
plt.xlabel("Timestamp")
plt.ylabel("Values")
plt.show()

In [ ]:
# Cleaning impact on ModA and ModB
if 'Cleaning' in cleaned.columns:
    impact = cleaned.groupby('Cleaning')[['ModA','ModB']].mean()
    impact.plot(kind='bar', figsize=(6,4))
    plt.title("Average ModA & ModB by Cleaning Flag")
    plt.show()
else:
    print("No Cleaning column found.")

In [ ]:
# Correlation heatmap
corr_cols = [c for c in ['GHI','DNI','DHI','TModA','TModB','ModA','ModB','Tamb','RH','WS','WSgust','BP'] if c in cleaned.columns]
plt.figure(figsize=(10,8))
sns.heatmap(cleaned[corr_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# Relationship analysis
pairs = [('WS','GHI'), ('RH','Tamb'), ('RH','GHI')]
for x,y in pairs:
    if x in cleaned.columns and y in cleaned.columns:
        plt.figure(figsize=(6,4))
        plt.scatter(cleaned[x], cleaned[y], s=10, alpha=0.5)
        plt.xlabel(x); plt.ylabel(y)
        plt.title(f"{y} vs {x}")
        plt.show()

In [ ]:
# Wind rose (approximate polar bar chart)
if 'WD' in cleaned.columns and 'WS' in cleaned.columns:
    bins = np.linspace(0, 360, 17)
    df_wind = cleaned[['WD','WS']].dropna().copy()
    df_wind['bin'] = pd.cut(df_wind['WD'], bins, include_lowest=True)
    rose = df_wind.groupby('bin')['WS'].mean()
    angles = np.linspace(0, 2*np.pi, len(rose), endpoint=False)

    fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(polar=True))
    ax.bar(angles, rose.values, width=2*np.pi/len(rose))
    ax.set_title("Wind Rose (Mean Wind Speed per Direction Bin)")
    plt.show()
else:
    print("Wind direction/speed data not found.")

In [ ]:
# Distribution analysis
for c in ['GHI','WS']:
    if c in cleaned.columns:
        plt.figure(figsize=(7,4))
        plt.hist(cleaned[c].dropna(), bins=40, color='skyblue')
        plt.title(f"Distribution of {c}")
        plt.xlabel(c); plt.ylabel("Frequency")
        plt.show()

In [ ]:
# Bubble chart: GHI vs Tamb (bubble size = RH or BP)
x, y = 'Tamb', 'GHI'
size = 'RH' if 'RH' in cleaned.columns else ('BP' if 'BP' in cleaned.columns else None)
if x in cleaned.columns and y in cleaned.columns and size:
    plt.figure(figsize=(8,6))
    plt.scatter(cleaned[x], cleaned[y], s=(cleaned[size]/cleaned[size].max())*200, alpha=0.5)
    plt.xlabel(x); plt.ylabel(y)
    plt.title(f"{y} vs {x} (bubble size = {size})")
    plt.show()
else:
    print("Missing required columns for bubble chart.")


## 🧾 Summary & Key Insights

**Key Findings:**
- Identified missing values and imputed them with median values.
- Outliers detected using Z-scores.
- Solar irradiance variables (GHI, DNI, DHI) strongly correlated.
- Cleaning events may impact module temperatures (`ModA`, `ModB`).
- Wind and humidity influence temperature and irradiance patterns.


